In [30]:
# Autoreload so we're using most recent modules
%load_ext autoreload
%autoreload 2

import sys
import os
import pandas as pd
import random

# Add the project root to the Python path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.dirname(notebook_dir)
sys.path.append(project_root)

# Import from src
from src.data.collection import (
    initialize_games_api,
    initialize_stats_api,
    initialize_teams_api,
    initialize_ratings_api,
    fetch_games,
    fetch_team_game_stats,
    get_games_df,
    get_team_game_stats_df,
    fetch_advanced_team_game_stats,
    fetch_team_talent,
    get_calendar,
    fetch_all_ratings
)
from pprint import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Inputs

In [2]:
start_year = 2001
end_year = 2023

## Games API

In [26]:
# Initialize the API
games_api = initialize_games_api()

# Fetch and store games data
print("Fetching games data...")
# fetch_games(start_year, end_year, games_api)

# Fetch and store team game stats data
print("Fetching team game stats data...")
# fetch_team_game_stats(start_year, end_year)

# Fetch and store calendar data
print("Fetching calendar data...")
calendar_data = get_calendar(2024, games_api)

Fetching games data...
Fetching team game stats data...
Fetching calendar data...


## Process Games

## Stats API

In [17]:
## Advanced Team Stats
stats_api = initialize_stats_api()
fetch_advanced_team_game_stats(2020, 2020, stats_api)

Successfully fetched advanced team game stats for 2020 regular season
Successfully fetched advanced team game stats for 2020 postseason season
Data appended in advanced_team_game_stats
Appended advanced team game stats data for year 2020
Finished fetching advanced team game stats data


## Ratings API

In [ ]:

ratings_api = initialize_ratings_api()
fetch_all_ratings(start_year, end_year, ratings_api)

## Teams API

In [21]:
## Team Talent Composite
teams_api = initialize_teams_api()
print("Fetching team talent data...")
fetch_team_talent(start_year, end_year, teams_api)

Fetching team talent data...
Data appended in team_talent
Appended team talent data for year 2020
Successfully fetched team talent data for 2020
Data appended in team_talent
Appended team talent data for year 2021
Successfully fetched team talent data for 2021
Data appended in team_talent
Appended team talent data for year 2022
Successfully fetched team talent data for 2022
Data appended in team_talent
Appended team talent data for year 2023
Successfully fetched team talent data for 2023
Data appended in team_talent
Appended team talent data for year 2024
Successfully fetched team talent data for 2024
Finished fetching team talent data
